In [6]:
import tensorflow as tf
from keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

In [7]:
# Directorio donde se encuentran las imágenes
data_dir = 'D:\\U\\7. Septimo\\RI\\ir24a\\week14\\caltech-101'

In [8]:
# Cargar las imágenes desde el directorio
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

Found 9144 files belonging to 102 classes.
Using 7316 files for training.
Found 9144 files belonging to 102 classes.
Using 1828 files for validation.


In [9]:
# Obtener el número de clases
class_names = train_dataset.class_names
num_classes = len(class_names)

In [10]:
# Preprocesar las imágenes (opcional, si es necesario)
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_dataset = train_dataset.map(preprocess_image)
test_dataset = test_dataset.map(preprocess_image)

# Cargar el modelo VGG16 con pesos preentrenados de ImageNet, sin la capa de clasificación superior
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Crear un nuevo modelo que produzca los mapas de características
model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)

58889256/58889256 [==============================] - 8s 0us/step


In [11]:
# Función para extraer características
def extract_features(dataset):
    features = []
    labels = []
    for images, lbls in dataset:
        feature_maps = model.predict(images)
        features.append(feature_maps)
        labels.append(lbls.numpy())
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

In [12]:
# Extraer características para los conjuntos de datos de entrenamiento y prueba
train_features, train_labels = extract_features(train_dataset)
test_features, test_labels = extract_features(test_dataset)

1/1 [==============================] - 1s 760ms/step


In [13]:
# Mostrar las dimensiones de los datos extraídos
print(f'Train features shape: {train_features.shape}')
print(f'Train labels shape: {train_labels.shape}')
print(f'Test features shape: {test_features.shape}')
print(f'Test labels shape: {test_labels.shape}')

Train features shape: (7316, 7, 7, 512)
Train labels shape: (7316,)
Test features shape: (1828, 7, 7, 512)
Test labels shape: (1828,)
